In [ ]:
pip install pandas

In [ ]:

pip install numpy


In [ ]:
pip install matplotlib


In [ ]:
pip install seaborn

In [ ]:
pip install scikit-learn


In [ ]:
pip install flask

In [ ]:
pip install joblib

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
import pandas as pd
import warnings

warnings.filterwarnings('ignore')


file_path = "/content/cmrit.bangalore_instagram_data.csv"

try:
    data = pd.read_csv(file_path, encoding='latin1')
    print("Data loaded successfully!")
    print(data.head())  # Display the first few rows of the dataset
except FileNotFoundError:
    print("Error: Dataset file not found. Please provide the correct file path.")
except UnicodeDecodeError as e:
    print(f"UnicodeDecodeError: {e}")
    print("Try using a different encoding like 'latin1' or 'ISO-8859-1'.")

print("All libraries loaded successfully!")


data['date'] = pd.to_datetime(data['date'], format='%d-%m-%Y %H:%M')
data['hour'] = data['date'].dt.hour
data['day'] = data['date'].dt.day_name()
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year
data['caption_length'] = data['caption'].apply(len)
data['num_hashtags'] = data['hashtags'].apply(lambda x: len(x.split()))


data['engagement'] = data['likes'] + data['comments'] + data['shares']


def categorize_engagement(engagement):
    if engagement < 50:  # Low engagement
        return 'low'
    elif 50 <= engagement < 100:  # Medium engagement
        return 'medium'
    else:  # High engagement
        return 'high'


data['engagement_category'] = data['engagement'].apply(categorize_engagement)


data = data.drop(columns=['date', 'post_url', 'shares', 'likes', 'comments', 'engagement'])


X = data.drop(columns=['engagement_category'])
y = data['engagement_category']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


numeric_features = ['hour', 'month', 'year', 'caption_length', 'num_hashtags']
categorical_features = ['day']
text_features = ['caption', 'hashtags']


preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features),
        ('text', TfidfVectorizer(), 'caption'),
        ('text2', TfidfVectorizer(), 'hashtags')
    ])


pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)


y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

# Example input for a new Instagram post prediction
new_data = pd.DataFrame({
    'caption': ['CMR Group of Institutions (CMRGI) hosted its annual Techno Meet for Society 2024 at the CMR Institute of Technology (CMRIT) on 23rd November 2024. This edition of Techno Meet for Society embraced the ethos of "Explore, Experience and Enjoy" in technology and innovation. During the exhibition, the teaching staff and students from various departments and schools under CMRIT presented innovative products and ideas, categorized into Startups, Hardware Projects, and Software Projects. The event showcased 34 projects and 2 product launches, including 18 software and 16 hardware projects, contributed by students and faculty. Visit our Website: https://www.cmr.edu.in/ For admissions: https://admissions.cmr.edu.in/'],
    'hashtags': ['#TechnoMeet2024, #InnovationForSociety, #CMRIT, #CMRGI, #ExploreExperienceEnjoy, #TechInnovation, #StudentProjects, #TechStartups, #HardwareProjects, #SoftwareProjects, #TechExhibition, #CMRUniversity, #EducationForExcellence, #FutureReady, #DigitalInnovation, #TechCommunity'],
    'hour': [12],  # Example hour feature
    'day': ['Monday'],  # Example day feature
    'month': [11],  # Example month feature
    'year': [2024],  # Example year feature
    'caption_length': [568],  # Example caption length
    'num_hashtags': [347]  # Example hashtags
})


predicted_engagement_category = pipeline.predict(new_data)
print(f"Predicted Engagement Category for the new post: {predicted_engagement_category[0]}")


Data loaded successfully!
               date                                            caption  \
0  06-12-2024 16:22  Department of Artificial Intelligence and Mach...   
1  06-12-2024 16:17  Codechef Club of the Department of Artificial ...   
2  05-12-2024 02:14  Department of Computer Science and Engineering...   
3  04-12-2024 12:34  Department of Electronics and Communication En...   
4  04-12-2024 11:56  Codechef Club of the Department of Artificial ...   

                                            hashtags  likes  comments  shares  \
0  #cmritbengaluru, #cmrit, #IIC_CMRIT, #mhrd, #m...     20         0       1   
1  #cmritbengaluru, #cmrit, #IIC_CMRIT, #mhrd, #m...     23         0       0   
2  #cmritbengaluru, #cmrit, #IIC_CMRIT, #mhrd, #m...     14         0       2   
3  #cmritbengaluru, #cmrit, #IIC_CMRIT, #mhrd, #m...     14         0       0   
4  #cmritbengaluru, #cmrit, #IIC_CMRIT, #mhrd, #m...     51         0       2   

                                          

In [ ]:
import joblib

# After training your model with pipeline
joblib.dump(pipeline, 'engagement_model.pkl')


['engagement_model.pkl']